In [18]:
import configparser

config = configparser.ConfigParser()
config.read('config.conf')
openai_api_key = config['Credentials']['OPENAI_API_KEY']


In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=openai_api_key
)

In [20]:
text = "This is a test query."
query_result = embeddings.embed_query(text)
print(query_result)

[-0.005047945771366358, 0.005088248755782843, -0.005195723380893469, -0.019143905490636826, -0.025538641959428787, 0.022824909538030624, -0.0143344197422266, -0.012023716233670712, -0.02723136730492115, -0.0352247878909111, 0.01058624405413866, 0.013259674422442913, 0.009249529801309109, -0.0004370352253317833, 0.002320778788998723, 0.0033652971033006907, 0.020742589607834816, -0.0037414582911878824, 0.023308545351028442, -0.01387765258550644, 0.000130354892462492, -0.004204942379146814, 0.0035970392636954784, 0.016591385006904602, -0.014132904820144176, -0.005709586199373007, 0.010149628855288029, -0.027338841930031776, 0.0032813327852636576, -0.0020319409668445587, 0.010720587335526943, 0.0014198396820574999, -0.00937715545296669, -0.02481319010257721, -0.016201790422201157, 0.005061380099505186, 0.005739813204854727, -0.005108400247991085, 0.025041572749614716, -0.0003772105264943093, 0.017625827342271805, 0.00898755993694067, -0.0027002983260899782, -0.017988555133342743, 0.0164839

In [21]:
len(query_result)

1536

In [22]:
import pandas as pd

df = pd.read_csv("noticias_lupa.csv")

In [24]:
df['RawText'].values

array(['\nRetrospectiva 2023\nLula e Bolsonaro são tema das verificações mais lidas do ano\n27.12.2023 - 08h00\nMaiquel Rosauro\nPorto Alegre - RS\nO presidente Luiz Inácio Lula da Silva (PT) e seu antecessor, Jair Bolsonaro (PL), estão relacionados ao conteúdo de sete das dez verificações mais lidas no site da Lupa em 2023. Entre as mais acessadas há também desinformações envolvendo golpes e desastres naturais. \nUma suposta arrecadação milionária em uma campanha, via Pix, para Bolsonaro foi a verificação mais lida do ano. Também figura entre as mais acessadas uma falsa crítica do “dono da Seara” aos bolsonaristas.\nLula, por sua vez, possui relação com metade das dez verificações mais lidas. Circularam ao longo de 2023 conteúdos desinformativos sobre o orçamento do Ministério da Agricultura, um suposto furto de canetas em reunião do G20 e a realização de um Feirão Limpa Nome pelo seu governo. Além disso, a esposa de Lula, Janja, e um dos filhos do petista figuraram entre as verificaç

In [29]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(data_frame=df, page_content_column = 'RawText')

docs = loader.load()

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

documents = text_splitter.split_documents(docs)

In [33]:
documents[0]

Document(page_content='Retrospectiva 2023\nLula e Bolsonaro são tema das verificações mais lidas do ano\n27.12.2023 - 08h00\nMaiquel Rosauro\nPorto Alegre - RS\nO presidente Luiz Inácio Lula da Silva (PT) e seu antecessor, Jair Bolsonaro (PL), estão relacionados ao conteúdo de sete das dez verificações mais lidas no site da Lupa em 2023. Entre as mais acessadas há também desinformações envolvendo golpes e desastres naturais. \nUma suposta arrecadação milionária em uma campanha, via Pix, para Bolsonaro foi a verificação mais lida do ano. Também figura entre as mais acessadas uma falsa crítica do “dono da Seara” aos bolsonaristas.', metadata={'Author': 'Por Maiquel Rosauro', 'Link': 'https://lupa.uol.com.br/jornalismo/2023/12/27/lula-e-bolsonaro-sao-tema-das-verificacoes-mais-lidas-do-ano', 'Title': 'Lula e Bolsonaro são tema das verificações mais lidas do ano', 'Headline': 'Levantamento da Lupa mostra as dez verificações mais lidas em 2023. Entre as campeãs, cinco possuem relação com Lu

In [34]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)

In [35]:
vectorstore

In [46]:
from langchain.prompts import PromptTemplate

prompt_template = """Você é um assistente útil para tirar dúvidas.

{context}

Pergunta: {question}
Resposta aqui:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [47]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

In [48]:
llm = OpenAI(openai_api_key=openai_api_key)

In [49]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
)

In [50]:
query = "Vacina funciona?"
result = qa.run(query)

In [51]:
result

'\n\nSim, as vacinas são eficazes em ajudar a prevenir doenças infecciosas. Estudos clínicos mostram que as vacinas são seguras e podem ajudar a evitar muitas doenças graves. Uma vez vacinado, o corpo é capaz de construir uma proteção contra a infecção. É importante manter-se atualizado com as vacinas recomendadas para a sua idade e situação.'

In [52]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True, output_key="answer"
)

In [53]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=openai_api_key),
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs={"prompt": PROMPT},
)

In [54]:
query = "Lula tirou 432 milhões da pobreza?"
qa({"question": query})
qa({"question": "E o Bolsonaro?"})

{'question': 'E o Bolsonaro?',
 'chat_history': [HumanMessage(content='Lula tirou 432 milhões da pobreza?', additional_kwargs={}, example=False),
  AIMessage(content='\n\nNão, Lula não tirou 432 milhões de pessoas da pobreza. O governo de Lula reduziu a pobreza extrema no Brasil por meio de programas de transferência de renda, mas o número de pessoas que saíram da pobreza durante seu governo estimado em cerca de 40 milhões.', additional_kwargs={}, example=False),
  HumanMessage(content='E o Bolsonaro?', additional_kwargs={}, example=False),
  AIMessage(content=' De acordo com dados do Instituto Brasileiro de Geografia e Estatística (IBGE), a população brasileira em situação de pobreza caiu de 39,1 milhões de pessoas em 2014 para 28,6 milhões de pessoas em 2019, enquanto o índice de extrema pobreza foi reduzido de 13,9 milhões de pessoas para 9,2 milhões durante o governo de Jair Bolsonaro.', additional_kwargs={}, example=False)],
 'answer': ' De acordo com dados do Instituto Brasileiro